In [1]:
import re
import json
import glob
import pickle
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [21]:
df = pd.read_csv("Clean_data/Kickstarter.csv")

In [22]:
urls = df.urls
ids = df.id

urls_ = list(map(lambda url:json.loads(url)["web"]["project"],urls))
urls = list(map(lambda url:url.replace("?ref=discovery_category_newest","/description"),urls_))

In [23]:
errors = np.zeros(len(df))
# storys = [""]*len(urls)

path = "Webdriver/chromedriver.exe"
options = webdriver.ChromeOptions()
options.add_argument('--lang=en')
# 0

In [ ]:
driver = webdriver.Chrome(executable_path = path,chrome_options=options)
for i in tqdm(range(1000,len(urls))):
    try:
        driver.get(urls[i])
        story_raw = WebDriverWait(driver,10).until(
        EC.presence_of_element_located((By.XPATH,"//div[contains(@class, 'rte__content')]"))
        )
        storys[i] = story_raw.text
    except:
        errors[i] = 1
    finally:
        time.sleep(np.random.randint(3000)/100)
driver.quit()

<ipython-input-24-04aa53824134>:1: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path = path,chrome_options=options)
 43%|████▎     | 1048/2429 [5:40:52<9:18:19, 24.26s/it] 

In [12]:
import requests
from bs4 import BeautifulSoup

s = requests.Session()
r = s.get("https://www.kickstarter.com")
soup = BeautifulSoup(r.text, 'html.parser')
xcsrf = soup.find("meta", {"name": "csrf-token"})["content"]

query = """
query GetEndedToLive($slug: String!) {
  project(slug: $slug) {
      id
      deadlineAt
      showCtaToLiveProjects
      state
      description
      url
      __typename
  }
}"""


r = s.post("https://www.kickstarter.com/graph",
    headers= {
        "x-csrf-token": xcsrf
    },
    json = {
        "query": query,
        "variables": {
            "slug":"kuhkubus-3d-escher-figures"
        }
    })

print(r.json())

query Campaign($slug: String!) {
  project(slug: $slug) {
    id
    isSharingProjectBudget
    risks
    story(assetWidth: 680)
    currency
    spreadsheet {
      displayMode
      public
      url
      data {
        name
        value
        phase
        rowNum
        __typename
      }
      dataLastUpdatedAt
      __typename
    }
    environmentalCommitments {
      id
      commitmentCategory
      description
      __typename
    }
    __typename
  }
}

query Campaign($slug: String!) {
  project(slug: $slug) {
    risks
    story(assetWidth: 680)
  }
}

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [17]:
import requests
from bs4 import BeautifulSoup
import re

urls = [ 
    "https://www.kickstarter.com/projects/1365297844/kuhkubus-3d-escher-figures?ref=discovery_staff_picks_category_newest",
    "https://www.kickstarter.com/projects/clarissaredwine/swingby-a-voyager-gravity-puzzle?ref=discovery_staff_picks_category_newest",
    "https://www.kickstarter.com/projects/100389301/us-army-navy-marines-air-force-special-challenge-c?ref=category"
]
slugs = []

#extract slugs from url
for url in urls:
    slugs.append(re.search('/projects/(.*)\?', url).group(1))

s = requests.Session()
r = s.get("https://www.kickstarter.com")
soup = BeautifulSoup(r.text, 'html.parser')
xcsrf = soup.find("meta", {"name": "csrf-token"})["content"]

query = """
query Campaign($slug: String!) {
  project(slug: $slug) {
    risks
    story(assetWidth: 680)
  }
}"""

for slug in slugs:
    print(f"--------{slug}------")
    r = s.post("https://www.kickstarter.com/graph",
        headers= {
            "x-csrf-token": xcsrf
        },
        json = {
            "operationName":"Campaign",
            "variables":{
                "slug": slug
            },
            "query": query
        })

    result = r.json()

    print("-------STORY--------")
    story_html = result["data"]["project"]["story"]
    soup = BeautifulSoup(story_html, 'html.parser')
    for i in soup.find_all('p'):
        print(i.get_text())

    print("-------RISKS--------")
    print(result["data"]["project"]["risks"])

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [78]:
num = 1000
sum(errors)

15.0

In [80]:
driver = webdriver.Chrome(executable_path = path,chrome_options=options)
for i in range(1000):
    if(errors[i] > 0):
        try:
            driver.get(urls[i])
            story_raw = WebDriverWait(driver,10).until(
            EC.presence_of_element_located((By.XPATH,"//div[contains(@class, 'rte__content')]"))
            )
            storys[i] = story_raw.text
            errors[i] = 0
        except:
            pass
        finally:
            time.sleep(np.random.randint(200)/100)
driver.quit()

<ipython-input-80-3bff04e2f4d2>:1: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path = path,chrome_options=options)


In [83]:
sum(errors)

0.0

In [84]:
df_export = pd.DataFrame({"id":df["id"],
                         "story":storys}, index = None)
df_export.to_csv("Output/Story.csv")